In [ ]:
# Initial setup and exploration
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Audio processing libraries
import librosa
import librosa.display
import soundfile as sf

print("🔧 Setting up development environment...")
print("✅ Core libraries imported")

# Check if our custom modules work
try:
    from src.audio_processing import AudioFeatureExtractor, AudioPreprocessor
    from src.models import BabyCryClassifier
    from src.context import BabyProfile, ContextManager
    from src.main import BabyWhisperClassifier
    print("✅ BabyWhisper modules imported successfully")
except Exception as e:
    print(f"❌ Module import error: {e}")

print("\n🍼 Ready to start developing BabyWhisper!")


In [ ]:
# Let's experiment with audio feature extraction
print("🎵 Exploring Audio Features for Baby Cry Classification")

# Initialize feature extractor
feature_extractor = AudioFeatureExtractor()

# Let's create some test audio to understand our features
print("\n📊 What audio features can we extract?")

# Create a simple synthetic cry for testing
duration = 3.0
sample_rate = 22050
t = np.linspace(0, duration, int(sample_rate * duration))

# Simulate a "hunger cry" - rhythmic pattern
freq_base = 300  # Base frequency
freq_variation = 50 * np.sin(2 * np.pi * 0.5 * t)  # Slow variation
cry_signal = np.sin(2 * np.pi * (freq_base + freq_variation) * t)

# Add rhythm (crying in bursts)
rhythm = (np.sin(2 * np.pi * 2 * t) > 0).astype(float)
cry_signal *= rhythm

# Add some decay and noise
cry_signal *= np.exp(-t * 0.3)
cry_signal += 0.05 * np.random.normal(0, 1, len(t))

# Normalize
cry_signal = cry_signal / np.max(np.abs(cry_signal)) * 0.8

print(f"📈 Created synthetic cry signal:")
print(f"   Duration: {duration}s")
print(f"   Sample rate: {sample_rate} Hz")
print(f"   Signal shape: {cry_signal.shape}")

# Save as temporary file for feature extraction testing
temp_file = 'temp_test_cry.wav'
sf.write(temp_file, cry_signal, sample_rate)
print(f"💾 Saved test audio: {temp_file}")

print("\n🔍 Testing feature extraction...")


In [ ]:
# Extract comprehensive features
try:
    features = feature_extractor.extract_features(temp_file)
    feature_vector = feature_extractor.extract_feature_vector(temp_file)
    
    print("✅ Feature extraction successful!")
    print(f"📊 Feature vector length: {len(feature_vector)}")
    print(f"🎯 Total features extracted: {len(feature_vector)}")
    
    # Let's examine what we got
    print("\n🔍 Feature breakdown:")
    print(f"   MFCC features: {len(features['mfcc_mean']) + len(features['mfcc_std'])} (mean + std)")
    print(f"   Spectral features: 6 (centroid, rolloff, bandwidth)")
    print(f"   Temporal features: 5 (ZCR, RMS, tempo)")
    print(f"   Chroma features: {len(features['chroma_mean']) + len(features['chroma_std'])} (mean + std)")
    print(f"   Mel-spectrogram: {len(features['mel_spec_mean']) + len(features['mel_spec_std'])} (mean + std)")
    print(f"   F0 & stats: 6 (fundamental freq + audio statistics)")
    
    # Show some actual values
    print(f"\n📈 Sample feature values:")
    print(f"   Spectral centroid: {features['spectral_centroid_mean']:.1f} Hz")
    print(f"   RMS energy: {features['rms_mean']:.3f}")
    print(f"   Zero crossing rate: {features['zcr_mean']:.3f}")
    print(f"   Duration: {features['duration']:.2f} seconds")
    print(f"   Max amplitude: {features['max_amplitude']:.3f}")
    
    # Clean up
    os.remove(temp_file)
    print(f"\n🧹 Cleaned up temporary file")
    
except Exception as e:
    print(f"❌ Feature extraction failed: {e}")
    if os.path.exists(temp_file):
        os.remove(temp_file)


In [ ]:
# Experiment with different model types
print("🧪 Testing Different Classification Models")

classifier = BabyCryClassifier()

# Let's see what models are available
print(f"📋 Available model types: {classifier.available_models}")

# Test with synthetic data first
from src.models.model_trainer import ModelTrainer
trainer = ModelTrainer()

print("\n🎲 Creating synthetic dataset for experimentation...")

# Generate small test dataset
features, labels = trainer.create_synthetic_dataset(n_samples=500)
print(f"✅ Generated {len(features)} samples with {features.shape[1]} features")
print(f"📊 Class distribution: {np.unique(labels, return_counts=True)}")

# Quick split for testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"\n📚 Training set: {X_train.shape}")
print(f"🧪 Test set: {X_test.shape}")

# Test individual models
models_to_test = ['random_forest', 'svm', 'neural_network']
results = {}

for model_type in models_to_test:
    print(f"\n🔬 Testing {model_type}...")
    try:
        # Initialize fresh classifier
        test_classifier = BabyCryClassifier(model_type=model_type)
        
        # Quick training
        test_classifier.train(X_train, y_train)
        
        # Test prediction
        accuracy = test_classifier.score(X_test, y_test)
        results[model_type] = accuracy
        
        print(f"   ✅ {model_type}: {accuracy:.3f} accuracy")
        
    except Exception as e:
        print(f"   ❌ {model_type} failed: {e}")
        results[model_type] = 0.0

print(f"\n🏆 Model Performance Summary:")
for model, score in results.items():
    print(f"   {model}: {score:.3f}")

best_model = max(results, key=results.get)
print(f"\n🥇 Best performing model: {best_model} ({results[best_model]:.3f})")


In [ ]:
# Experiment with Context Intelligence
print("🧠 Testing Context-Aware Prediction Layer")

# Create baby profiles for testing
now = datetime.now()

# Different scenarios to test
baby_scenarios = {
    "Emma (Recently Fed)": {
        "profile": BabyProfile("Emma", age_months=3),
        "last_feeding": now - timedelta(minutes=20),  # Just fed
        "last_sleep": now - timedelta(hours=1),
        "last_diaper": now - timedelta(minutes=30)
    },
    
    "Oliver (Very Hungry)": {
        "profile": BabyProfile("Oliver", age_months=6),
        "last_feeding": now - timedelta(hours=4.5),  # Long time!
        "last_sleep": now - timedelta(hours=2),
        "last_diaper": now - timedelta(hours=1)
    },
    
    "Newborn Lily (Tired)": {
        "profile": BabyProfile("Lily", age_months=1),
        "last_feeding": now - timedelta(hours=1.5),
        "last_sleep": now - timedelta(hours=2.5),  # Tired newborn
        "last_diaper": now - timedelta(hours=3)
    }
}

# Set up profiles
for name, scenario in baby_scenarios.items():
    profile = scenario["profile"]
    profile.update_feeding(scenario["last_feeding"])
    profile.update_sleep(sleep_end=scenario["last_sleep"])
    profile.update_diaper_change(scenario["last_diaper"])
    
    print(f"\n👶 {name}:")
    print(f"   🍼 Hours since feeding: {profile.get_time_since_feeding():.1f}")
    print(f"   💤 Hours awake: {profile.get_time_awake():.1f}")
    print(f"   🧷 Hours since diaper: {profile.get_time_since_diaper_change():.1f}")
    print(f"   📊 Context suggests - Hungry: {profile.is_likely_hungry()}, Tired: {profile.is_likely_tired()}")

print("\n" + "="*50)


In [ ]:
# Test how context changes AI predictions
context_manager = ContextManager()
classes = ['hunger', 'pain', 'discomfort', 'tiredness', 'normal']

# Simulate different AI base predictions
test_scenarios = [
    {
        "name": "AI says: HUNGER cry", 
        "base_probs": np.array([0.75, 0.10, 0.08, 0.05, 0.02])  # Strong hunger prediction
    },
    {
        "name": "AI says: DISCOMFORT cry",
        "base_probs": np.array([0.20, 0.15, 0.55, 0.08, 0.02])  # Moderate discomfort
    }
]

print("🔄 How Context Changes AI Predictions:\n")

for scenario in test_scenarios:
    print(f"🎵 {scenario['name']}")
    base_prediction = classes[np.argmax(scenario['base_probs'])]
    base_confidence = np.max(scenario['base_probs'])
    print(f"   🤖 Base AI: {base_prediction} ({base_confidence:.2f})")
    
    # Test with different baby contexts
    for baby_name, baby_data in baby_scenarios.items():
        profile = baby_data["profile"]
        
        # Apply context intelligence
        result = context_manager.apply_context_to_prediction(
            scenario['base_probs'], profile, classes
        )
        
        context_pred = result['context_adjusted_prediction']
        context_conf = result['context_adjusted_confidence']
        changed = "🔄" if result['prediction_changed'] else "✓"
        
        print(f"   👶 {baby_name}: {context_pred} ({context_conf:.2f}) {changed}")
    
    print()  # Empty line between scenarios

print("💡 Key Insight: Same cry sounds → Different conclusions based on baby context!")
print("This is what makes BabyWhisper intelligent, not just a classifier.")


In [ ]:
# Full System Integration Test
print("🍼 BABYWHISPER FULL SYSTEM TEST 🍼")
print("="*50)

# Initialize the complete system
baby_whisper = BabyWhisperClassifier()

# Check system status
print("🔧 System Status Check:")
status = baby_whisper.get_system_status()
for key, value in status.items():
    print(f"   {key}: {value}")

# Train model if needed
if not status['model_loaded']:
    print("\n🎓 Training model...")
    baby_whisper.train_new_model(use_synthetic_data=True)
    print("   ✅ Model training completed!")

# Create a test baby profile
print(f"\n👶 Creating baby profile...")
profile_id = baby_whisper.create_baby_profile(
    name="Test Baby Emma",
    age_months=4,
    birth_date=datetime.now() - timedelta(days=4*30)
)

# Update baby context
baby_whisper.update_baby_context(
    profile_id=profile_id,
    feeding_time=datetime.now() - timedelta(hours=2.5),
    nap_time=datetime.now() - timedelta(hours=1.5),
    diaper_change_time=datetime.now() - timedelta(minutes=45)
)

print(f"   ✅ Baby profile created: {profile_id}")

# Get baby insights
insights = baby_whisper.get_baby_insights(profile_id)
print(f"\n📊 Baby Insights:")
for key, value in insights.items():
    if isinstance(value, list):
        print(f"   {key}: {', '.join(value) if value else 'None'}")
    else:
        print(f"   {key}: {value}")

print(f"\n🎉 SYSTEM INTEGRATION SUCCESSFUL!")
print(f"   ✅ All components working together")
print(f"   ✅ Context-aware intelligence operational") 
print(f"   ✅ Ready for real-world testing!")


In [ ]:
# Quick Demo: BabyWhisper in Action! 
print("🎬 QUICK DEMO: BabyWhisper Intelligence in Action!")
print("="*60)

# Simulate different babies and scenarios
demo_cases = [
    {
        "baby": "Emma (3 months, just fed 20 min ago)",
        "ai_prediction": "hunger", 
        "ai_confidence": 0.78,
        "context_result": "discomfort",
        "context_confidence": 0.62,
        "explanation": "Baby unlikely to be hungry so soon after feeding. Check diaper or comfort needs."
    },
    {
        "baby": "Oliver (6 months, fed 4 hours ago)", 
        "ai_prediction": "tiredness",
        "ai_confidence": 0.65,
        "context_result": "hunger", 
        "context_confidence": 0.82,
        "explanation": "Despite cry patterns suggesting tiredness, baby is overdue for feeding. Try feeding first."
    },
    {
        "baby": "Newborn Lily (awake 2.5 hours)",
        "ai_prediction": "pain",
        "ai_confidence": 0.70, 
        "context_result": "tiredness",
        "context_confidence": 0.85,
        "explanation": "Newborns get overtired quickly. Cry intensity might be from exhaustion, not pain."
    }
]

for i, case in enumerate(demo_cases, 1):
    print(f"\n📱 Case {i}: {case['baby']}")
    print(f"   🤖 AI says: {case['ai_prediction']} ({case['ai_confidence']:.2f})")
    print(f"   🧠 Context says: {case['context_result']} ({case['context_confidence']:.2f})")
    print(f"   💡 Smart insight: {case['explanation']}")

print(f"\n🌟 This is what makes BabyWhisper special:")
print(f"   🎯 Not just classification - intelligent interpretation")
print(f"   👶 Understands each baby's unique patterns") 
print(f"   🧠 Combines AI with real-world parenting knowledge")
print(f"   💕 Helps exhausted parents make better decisions")

print(f"\n🍼 BabyWhisper: Where AI meets parental intuition! ✨")


In [ ]:
# Setup and Installation
import sys
import os
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("BabyWhisper Demo Notebook - Ready to start!")
